## Preliminaries

In [1]:
library(ape)
library(ggtree)
library(phangorn)
library(Quartet)
library(adephylo)
library(ggplot2)
library(reshape2)

Registered S3 method overwritten by 'ggtree':
  method      from 
  identify.gg ggfun

ggtree v3.2.1  For help: https://yulab-smu.top/treedata-book/

If you use ggtree in published research, please cite the most appropriate paper(s):

1. Guangchuang Yu. Using ggtree to visualize data on tree-like structures. Current Protocols in Bioinformatics. 2020, 69:e96. doi:10.1002/cpbi.96
2. Guangchuang Yu, Tommy Tsan-Yuk Lam, Huachen Zhu, Yi Guan. Two methods for mapping and visualizing associated data on phylogeny using ggtree. Molecular Biology and Evolution. 2018, 35(12):3041-3043. doi:10.1093/molbev/msy194
3. Guangchuang Yu, David Smith, Huachen Zhu, Yi Guan, Tommy Tsan-Yuk Lam. ggtree: an R package for visualization and annotation of phylogenetic trees with their covariates and other associated data. Methods in Ecology and Evolution. 2017, 8(1):28-36. doi:10.1111/2041-210X.12628




Attaching package: ‘ggtree’


The following object is masked from ‘package:ape’:

    rotate


Loading requir

#### Function for displaying face-to-face plot

In [ ]:
plot_cophylo <- function(tree1, tree2) {
    cophyloplot(ladderize(tree1), ladderize(tree2),
                assoc = matrix(rep(tree2$tip.label, 2), ncol = 2),
                length.line = 4, space = 28, gap = 3);
}

#### Function to display RF and Quartet metrics

In [ ]:
display_metrics <- function(tree, ref_tree) {
    splitStatuses <- SplitStatus(tree, ref_tree)
    quartetStatuses <- QuartetStatus(tree, ref_tree)
    # the least, the better (by distance)
    print(paste("Robinson-Foulds: ", RawSymmetricDifference(splitStatuses)))
    print(paste("Robinson-Foulds (normalized)", RawSymmetricDifference(splitStatuses) / splitStatuses[, 'N']))
    # the greater, the better (by similarity)
    print(paste("Similarity by Quartets: ", SymmetricDifference(quartetStatuses)))
}

## Plotting and comparison of trees

#### Loading reference tree

In [2]:
ref_tree <- read.tree("dummy_tree.nwk")

#### Rank vs. Reference

In [ ]:
rank_tree <- read.tree("rank/rank_tree.nwk")

display_metrics(rank_tree, ref_tree)

#png("ref_vs_rank.png")
plot_cophylo(ref_tree, rank_tree)
#dev.off()

In [3]:
confs <- c("GXIEZ","GXIEz","GXIeZ","GXIez","GXiEZ","GXiEz","GXieZ","GXiez",
           "GxIEZ","GxIEz","GxIeZ","GxIez","GxiEZ","GxiEz","GxieZ","Gxiez",
           "gXIEZ","gXIEz","gXIeZ","gXIez","gXiEZ","gXiEz","gXieZ","gXiez",
           "gxIEZ","gxIEz","gxIeZ","gxIez","gxiEZ","gxiEz","gxieZ","gxiez")

df_quartet <- setNames(data.frame(matrix(ncol = length(confs), nrow = 10)), confs)
df_rf <- setNames(data.frame(matrix(ncol = length(confs), nrow = 10)), confs)

for (conf in confs) {
    quartets <- c()
    rfs <- c()
    for (i in 1:10) {
        rank_tree <- read.tree(paste("rank/rank_tree_", conf, "_iter", i, ".nwk", sep=""))
        
        quartetStatuses <- QuartetStatus(rank_tree, ref_tree)
        quartet <- SymmetricDifference(quartetStatuses)
        quartets <- c(quartets, quartet)
        
        splitStatuses <- SplitStatus(rank_tree, ref_tree)
        rf <- RawSymmetricDifference(splitStatuses) / splitStatuses[, 'N']
        rfs <- c(rfs, rf)
    }
    df_quartet[conf] <- quartets
    df_rf[conf] <- rfs
}

png("ref-vs-rank_quartet.png")
ggplot(melt(df_quartet, id.vars = NULL, value.name = "quartet", variable.name = "parameters"), aes(x=parameters,y=quartet)) +
    geom_boxplot() +
    facet_wrap(.~parameters, ncol = 8, scales = "free")
dev.off()

png("ref-vs-rank_rf.png")
ggplot(melt(df_rf, id.vars = NULL, value.name = "RF", variable.name = "parameters"), aes(x=parameters,y=RF)) +
    geom_boxplot() +
    facet_wrap(.~parameters, ncol = 8, scales = "free")
dev.off()

png 
  2

png 
  2

#### Rank-Indel vs. Reference

In [ ]:
rankindl_tree <- read.tree("rank/rankindl_tree.nwk")

display_metrics(rankindl_tree, ref_tree)

#png("ref_vs_rankindl.png")
plot_cophylo(ref_tree, rankindl_tree)
#dev.off()

#### DCJ vs. Reference

In [ ]:
dcj_tree <- read.tree("dcj/dcj_tree.nwk")

display_metrics(dcj_tree, ref_tree)

#png("ref_vs_dcj.png")
plot_cophylo(ref_tree, dcj_tree)
#dev.off()